In [1]:
%load_ext autoreload
%autoreload 2
import ase
from ase.visualize import view
from ase.build import molecule
from ase.io import Trajectory, write, read
from ase.units import mol # Avogadro Constant
# calculator
import pfp_api_client
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
# Molecular Dynamics
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary
from ase.units import kB, fs, Pascal

import numpy as np

def vis(atoms):
    v = view(atoms, viewer='ngl')
    v.view.add_representation("ball+stick")
    display(v)

#estimator = Estimator(calc_mode='CRYSTAL_PLUS_D3')
estimator = Estimator(model_version="v2.0.0", max_retries=30,  calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3)
calculator = ASECalculator(estimator)

from ase.constraints import FixAtoms

print(f"pfp_api_client: {pfp_api_client.__version__}")
print(f"Estimator model version: {estimator.model_version}")
print("available versions: ", estimator.available_models)

pfp_api_client: 1.15.0
Estimator model version: v2.0.0
available versions:  ['latest', 'v0.0.0', 'v1.0.0', 'v1.1.0', 'v2.0.0', 'v3.0.0', 'v4.0.0', 'v5.0.0', 'v6.0.0']


In [2]:
import os
import sys
sys.path.append(os.path.join('..'))

from lammpslib3 import LAMMPSlib
#from ase.calculators.mixing import SumCalculator
from mixing_mod import SumCalculator

# Example of hybrid/overlay

cmds = ["pair_style hybrid/overlay lj/cut 12.0 lj/cut/coul/long 12.0 12.0 airebo 3.0",
        "pair_coeff * * airebo CH.airebo C NULL NULL",
        "pair_coeff * * lj/cut 0 0.1",
        "pair_coeff 1 3 lj/cut 0.0032 3.190",
        "pair_coeff * * lj/cut/coul/long 0 0.1",
        "kspace_style ewald 1.0e-4",]

# Long range for water
cmdLR = ["pair_style hybrid/overlay lj/cut 12.0 lj/cut/coul/long 20.0 20.0",
         "pair_coeff * * lj/cut 0 0",
         "pair_coeff 1 1 lj/cut/coul/long 0 0",
         "kspace_style ewald 1.0e-4",]

lammpsLR = LAMMPSlib(lmpcmds=cmdLR,lammps_header=['units metal', 'atom_style charge', 'atom_modify map array sort 0 0',],
                   atom_types={'C':1, 'H':2, 'O':3}, keep_alive=True)

lammps1 = LAMMPSlib(lmpcmds=cmds,lammps_header=['units metal', 'atom_style charge', 'atom_modify map array sort 0 0'],
                   atom_types={'C':1, 'H':2, 'O':3}, keep_alive=True)
lammps2 = LAMMPSlib(lmpcmds=cmds,lammps_header=['units metal', 'atom_style charge', 'atom_modify map array sort 0 0'],
                   atom_types={'C':1, 'H':2, 'O':3}, keep_alive=True)


In [3]:
from simpleQMMM_lammps_charge import SimpleQMMM_charge

atoms = read("NVT_charge_init.traj")
CHARGE_VALUE=0.01
#CHARGE_VALUE=0.00

atnum = atoms.get_atomic_numbers()
#print(atnum)
qm_idx = np.arange(len(atoms))
qm_idx = qm_idx[atnum != 6]
charge_idx=range(144)
charge_values = np.zeros_like(charge_idx, dtype="float")
charge_values[:72] = CHARGE_VALUE
charge_values[72:] = -CHARGE_VALUE

pfp_plus_LR = SumCalculator([calculator, lammpsLR], ignore_not_implemented=True)

atoms.set_calculator( SimpleQMMM_charge(qm_idx,
                    pfp_plus_LR,
                    lammps1,
                    lammps2,
                    charge_idx,
                    charge_values,
                    vacuum=None,  # if None, QM cell = MM cell
                    ) )

print(atoms.get_potential_energy())
np.savetxt("force_ewald_check.txt",atoms.get_forces())

-5747.135779319401


In [4]:
posz = atoms.get_positions()[:,2]
Natoms=posz.shape[0]
print(Natoms)
mask0 = np.arange(Natoms)
mask = ( mask0 >= 984 )
#print(mask, np.sum(mask), 3*16)
constraint = FixAtoms(mask=mask)
atoms.set_constraint(constraint) # set constraint

atoms.wrap()

cell0 = atoms.get_cell()

atoms = read("NVT_charge_init.traj")
atoms.set_calculator( SimpleQMMM_charge(qm_idx,
                    pfp_plus_LR,
                    lammps1,
                    lammps2,
                    charge_idx,
                    charge_values,
                    vacuum=None,  # if None, QM cell = MM cell
                    ) )
print(atoms.get_potential_energy())
vis(atoms)

1128
-5747.135779319401


In [5]:
#from ase.md.npt import NPT
from ase.md import Langevin
import time
time_sta = time.perf_counter()
time_pre = time_sta

#lmp = lammps()

OUT_traj = "NVT_charge.traj"
OUT_log = "NVT_charge.log"

T = 350.0
#P = 100000*Pascal
dt = 1.0*fs

#tdamp = 100*dt
#pdamp = pdamp = 0.6*(75*fs)**2.0 


MaxwellBoltzmannDistribution(atoms, temperature_K = T)
Stationary(atoms)
atoms.get_temperature()

#dyn = NPT(atoms, dt, temperature_K = T, externalstress = P, ttime = tdamp, pfactor = pdamp,  trajectory = OUT_traj, mask=[[1,0,0],[0,1,0],[0,0,1]], loginterval=100)

dyn = Langevin(atoms=atoms, timestep=dt, 
               temperature_K = T, friction=0.01, trajectory = OUT_traj, loginterval=100)


Header= "step etotal T e_pot stress"
print("NVT Simulation")
print(Header)

weight = np.sum(atoms.get_masses())*1.673e-27

file = open(OUT_log, "w")
file.write(Header + "\n")
def print_dyn():
    global time_pre
    #V = atoms.get_volume()*1E-30 # in m^3
    #density = weight/V
    time_now = time.perf_counter()
    line = f"{dyn.get_number_of_steps(): >3} {atoms.get_total_energy():.6f} {atoms.get_temperature():.6f} {atoms.get_potential_energy():.6f} {-atoms.get_stress(include_ideal_gas=True)[0:3].sum()/(3.0*Pascal):.6f} {time_now-time_sta:.3f} {time_now-time_pre:.3f}"
    #line = f"{dyn.get_number_of_steps(): >3} {atoms.get_total_energy():.6f} {atoms.get_temperature():.6f} {atoms.get_potential_energy():.6f} {-atoms.get_stress(include_ideal_gas=True)[0:3].sum()/(3.0*Pascal):.6f} {V:8.6e} {density:8.6e}"
    time_pre = time_now
    file.write(line+"\n")
    print(line, end = "\r")
    
    
dyn.attach(print_dyn, interval=50)

#steps = 1000000
steps = 500
dyn.run(steps)

file.close()

NVT Simulation
step etotal T e_pot stress


In [6]:
# test visualization

from ase.io import read,write

OUT_traj = "NVT_charge.traj"
test_traj = read(OUT_traj, index=":")
#test_traj = read(OUT_traj, index=":")
out_xyz = "NVT_charge.xyz"

for iframe, atoms in enumerate(test_traj):
    
    if iframe ==0:
        atoms.wrap()
        write(out_xyz, atoms)
    else:
        write(out_xyz, atoms, append=True)        